# Atelier sur la génétique statistique et l'épidémiologie génétique STAGE-Québec
## Thème 2 - Phénotypes moléculaires en épidémiologie génétique

Par Marc-André Legault (Université de Montréal) et Qihuang Zhang (Université McGill)

**31 juillet - 1er août 2025**

### Comparaison des méthodologies TWAS : S-PrediXcan vs FUSION

Ce carnet présente une analyse comparative examinant comment différentes méthodes TWAS (Transcriptome-Wide Association Study) peuvent donner des résultats variés quand appliquées au même jeu de données. Comprendre ces différences méthodologiques est essentiel pour interpréter les découvertes TWAS et sélectionner des approches analytiques appropriées pour les applications de recherche.

**Approche analytique :**
Nous avons mené des analyses TWAS parallèles des mêmes données d'expression génique pancréatique en utilisant deux méthodes établies :
- **S-PrediXcan** : Employant des modèles Elastic Net avec ajustement de polygénicité
- **FUSION** : Nous nous concentrons sur le modèle Elastic Net inclus pour une comparaison directe

**Objectifs d'apprentissage :**
À la fin de ce carnet, les participants comprendront :
- Comment différentes méthodes TWAS peuvent produire des résultats différents à partir de données identiques
- L'impact de la calibration de polygénicité sur la signification statistique
- Les critères pour évaluer la fiabilité de différentes approches
- Comment interpréter les résultats TWAS comparatifs dans des contextes de recherche

Les fichiers de résultats des analyses précédentes sont stockés dans le répertoire checkpoints et peuvent être utilisés pour exécuter ce carnet.

In [ ]:
library(ggplot2)

## Chargement et préparation des données

Nous allons maintenant charger et préparer les résultats des deux approches TWAS pour une comparaison directe. Ce processus implique :

1. **Chargement des résultats S-PrediXcan** : Incluant les statistiques calibrées et non calibrées
2. **Chargement des résultats FUSION** : Filtrage pour les modèles Elastic Net pour correspondre à l'approche de S-PrediXcan
3. **Fusion des jeux de données** : Combinaison des résultats par identifiant de gène pour l'analyse comparative

Nous commençons par charger la bibliothèque de visualisation nécessaire et examiner nos données :

In [ ]:
df_predixcan <- read.csv("results/twas_en_Pancreas.csv")

# Nous gardons les statistiques TWAS non calibrées dans un DF séparé.
df_predixcan_uncal <- df_predixcan[, c("gene", "gene_name", "uncalibrated_zscore", "uncalibrated_pvalue", "pred_perf_r2")]
names(df_predixcan_uncal) <- c("ENSG", "gene_label", "Z_PREDIXCAN_NO_CALIB", "P_PREDIXCAN_NO_CALIB", "R2_PREDIXCAN")

df_predixcan <- df_predixcan[, c("gene", "gene_name", "zscore", "pvalue", "pred_perf_r2")]
names(df_predixcan) <- c("ENSG", "gene_label", "Z_PREDIXCAN", "P_PREDIXCAN", "R2_PREDIXCAN")
nrow(df_predixcan)

In [ ]:
df_fusion <- read.csv(
    "results/fusion_GTEx_pancreas.tsv",
    sep="\t",
    stringsAsFactors=F
)
# Garder seulement le modèle "enet" qui est la même approche de régression pénalisée
# que pour S-PrediXcan pour maximiser la comparabilité.
df_fusion <- df_fusion[df_fusion$MODEL == "enet", ]
df_fusion <- df_fusion[, c("ID", "TWAS.Z", "TWAS.P", "MODELCV.R2")]
df_fusion <- df_fusion[complete.cases(df_fusion), ]
names(df_fusion) <- c("ENSG", "Z_FUSION", "P_FUSION", "R2_FUSION")
df_fusion$P_FUSION <- as.numeric(df_fusion$P_FUSION)
df_fusion$Z_FUSION <- as.numeric(df_fusion$Z_FUSION)

nrow(df_fusion)

In [ ]:
merged_uncalibrated <- merge(df_predixcan_uncal, df_fusion, by="ENSG")
merged <- merge(df_predixcan, df_fusion, by="ENSG")
nrow(merged)

## Analyse comparative : Résultats non calibrés

Nous commençons notre comparaison avec les résultats S-PrediXcan **non calibrés** versus FUSION. Cette comparaison est particulièrement informative car elle démontre la sortie statistique brute avant tout ajustement de polygénicité.

**Aspects clés de cette comparaison :**
- **Seuil de signification** : Correction de Bonferroni appliquée (P ≤ 0,05 / 1 058 gènes)
- **Régions codées par couleur** : Le graphique divise les résultats en trois zones interprétatives
- **Comparaison méthodologique directe** : Les deux méthodes utilisent des modèles Elastic Net sous-jacents identiques

La visualisation ci-dessous illustre où les méthodes s'accordent, divergent, et les implications pour l'interprétation biologique :

In [ ]:
p_thresh <- -log10(0.05 / 1058)

ggplot(merged_uncalibrated, aes(x = -log10(P_FUSION), y = -log10(P_PREDIXCAN_NO_CALIB))) +
    geom_rect(aes(xmin = -Inf, xmax = p_thresh, ymin = p_thresh, ymax = Inf), 
              fill = "#eff5ff", inherit.aes = FALSE) +
    geom_rect(aes(xmin = p_thresh, xmax = Inf, ymin = p_thresh, ymax = Inf), 
              fill = "#ffeff5", inherit.aes = FALSE) +
    geom_rect(aes(xmin = p_thresh, xmax = Inf, ymin = -Inf, ymax = p_thresh), 
              fill = "#f5ffef", inherit.aes = FALSE) +
    geom_point() +
    geom_vline(xintercept = -log10(0.05 / 1058), linetype = "dotted", color = "#888888", size = 1) +
    geom_hline(yintercept = -log10(0.05 / 1058), linetype = "dotted", color = "#888888", size = 1) +
    geom_abline(intercept = 0, slope = 1, linetype = "dashed", color = "red") +
    theme_bw()

<div class="alert alert-info">    
<p><strong>Examinez les trois régions colorées dans le graphique ci-dessus :</strong></p>
    
<p><strong>Région bleue (haut-gauche) :</strong> Gènes significatifs seulement par S-PrediXcan
    <br /><strong>Région rose (haut-droite) :</strong> Gènes significatifs par les deux méthodes  
    <br /><strong>Région verte (bas-droite) :</strong> Gènes significatifs seulement par FUSION</p>
    <p>
    <strong>Questions de discussion :</strong>
    <ul>
        <li>Quelles sont les implications quand un gène tombe dans chaque région colorée ?</li>
        <li>Pourquoi deux méthodes utilisant le même modèle sous-jacent pourraient-elles produire des résultats différents ?</li>
        <li>Quels gènes fourniraient la preuve la plus forte pour des études de suivi ?</li>
        <li>Comment les gènes peuvent-ils être significatifs dans une méthode mais pas l'autre ?</li>
    </ul>
    </p>
    <p><strong>Exercice d'exploration de données :</strong> Utilisez la cellule d'exploration de données ci-dessous pour identifier des exemples spécifiques :</p>
    <ul>
        <li>Identifier un gène significatif dans les deux approches (région rose)</li>
        <li>Trouver un gène significatif seulement dans FUSION (région verte)</li>
        <li>Localiser un gène significatif seulement dans S-PrediXcan (région bleue)</li>
    </ul>
    <p><em>Considérez examiner les gènes avec les différences les plus extrêmes entre les méthodes et leur pertinence biologique potentielle au diabète.</em></p>
</div>

In [ ]:
# Vous pouvez utiliser cette cellule pour explorer les données !

## L'impact de la calibration de polygénicité

Nous examinons maintenant la comparaison entre **S-PrediXcan calibré versus FUSION**. Cette analyse révèle l'impact pratique de l'ajustement de polygénicité de S-PrediXcan—un développement méthodologique conçu pour tenir compte de la nature polygénique des traits complexes.

**Méthodologie de calibration de polygénicité :**
S-PrediXcan ajuste les statistiques de test pour tenir compte de l'architecture polygénique des traits complexes, où plusieurs gènes influencent simultanément des phénotypes tels que le diabète. Cette calibration peut substantiellement modifier quels gènes apparaissent statistiquement significatifs et vise à tenir compte des associations fallacieuses.

**Effets attendus :**
La calibration résulte typiquement en des inférences statistiques plus conservatrices dans S-PrediXcan, réduisant potentiellement les associations faussement positives tout en augmentant possiblement les faux négatifs. La comparaison ci-dessous démontre comment cet ajustement affecte nos résultats d'association au diabète.

L'analyse suivante examine comment la calibration transforme nos résultats :

In [ ]:
p_thresh <- -log10(0.05 / 1058)

ggplot(merged, aes(x = -log10(P_FUSION), y = -log10(P_PREDIXCAN))) +
    geom_rect(aes(xmin = -Inf, xmax = p_thresh, ymin = p_thresh, ymax = Inf), 
              fill = "#eff5ff", inherit.aes = FALSE) +
    geom_rect(aes(xmin = p_thresh, xmax = Inf, ymin = p_thresh, ymax = Inf), 
              fill = "#ffeff5", inherit.aes = FALSE) +
    geom_rect(aes(xmin = p_thresh, xmax = Inf, ymin = -Inf, ymax = p_thresh), 
              fill = "#f5ffef", inherit.aes = FALSE) +
    geom_point() +
    geom_vline(xintercept = p_thresh, linetype = "dotted", color = "#888888", size = 1) +
    geom_hline(yintercept = p_thresh, linetype = "dotted", color = "#888888", size = 1) +
    geom_abline(intercept = 0, slope = 1, linetype = "dashed", color = "red") +
    theme_bw()

<div class="alert alert-info">
    <h4>Questions analytiques clés :</h4>
    <ul>
        <li><strong>Ampleur de l'effet :</strong> Dans quelle mesure l'ajustement de polygénicité modifie-t-il les valeurs p ?</li>
        <li><strong>Patron de changement :</strong> Tous les gènes sont-ils affectés uniformément, ou certains montrent-ils des changements plus substantiels ?</li>
        <li><strong>Fiabilité méthodologique :</strong> Étant donné ces différences, quelle approche fournit une identification plus fiable des vraies associations gène-trait ?</li>
        <li><strong>Considérations pratiques :</strong> Comment ces différences devraient-elles informer la sélection de méthodologie TWAS ?</li>
    </ul>
</div>

Répondez ici ! Vous pouvez aussi utiliser une autre cellule de code pour explorer les données.